***
# 0. Load the dependencies
***

In [1]:
!pip install open3d
!pip install learning3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
     ━━

In [146]:
!pip install gdown

In [147]:
!gdown 14TKpL0Mi24EC39B0-DInI7ccdc_v_k5d

Downloading...
From (original): https://drive.google.com/uc?id=14TKpL0Mi24EC39B0-DInI7ccdc_v_k5d
From (redirected): https://drive.google.com/uc?id=14TKpL0Mi24EC39B0-DInI7ccdc_v_k5d&confirm=t&uuid=55cd1363-1cfc-4295-a72f-4ce5c399c55f
To: /kaggle/working/test_retrieval.zip
100%|█████████████████████████████████████████| 649M/649M [00:02<00:00, 235MB/s]


In [148]:
!unzip /kaggle/working/test_retrieval.zip -d /kaggle/working/


Archive:  /kaggle/working/test_retrieval.zip
   creating: /kaggle/working/test_retrieval/
   creating: /kaggle/working/test_retrieval/query/
  inflating: /kaggle/working/test_retrieval/query/0.png  
  inflating: /kaggle/working/test_retrieval/query/1.png  
  inflating: /kaggle/working/test_retrieval/query/2.png  
  inflating: /kaggle/working/test_retrieval/query/3.png  
  inflating: /kaggle/working/test_retrieval/query/4.png  
  inflating: /kaggle/working/test_retrieval/query/5.png  
  inflating: /kaggle/working/test_retrieval/query/6.png  
  inflating: /kaggle/working/test_retrieval/query/7.png  
  inflating: /kaggle/working/test_retrieval/query/8.png  
  inflating: /kaggle/working/test_retrieval/query/9.png  
  inflating: /kaggle/working/test_retrieval/query/10.png  
  inflating: /kaggle/working/test_retrieval/query/11.png  
  inflating: /kaggle/working/test_retrieval/query/12.png  
  inflating: /kaggle/working/test_retrieval/query/13.png  
  inflating: /kaggle/working/test_retrieval

In [152]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

from timm.data import create_transform
import timm
import os 
from tqdm import tqdm

import open3d as o3d
from learning3d.models import PointNet, DGCNN, PPFNet, DCP
from learning3d.models import Classifier

***
# 1. Prediction
***

## 1.1 Load the database

In [149]:
%%time

# Load all the database into the RAM, in form of a dictionary
db_path = '/kaggle/working/test_retrieval/database'

db_names = os.listdir(db_path)
#db_list = ['/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/database/' + path for path in db_names]
database_arr = {}

for path in db_names:
    mesh = o3d.io.read_triangle_mesh(db_path + '/' + path)
    pcd = mesh.sample_points_uniformly(1024)
    point_arr = np.asarray(pcd.points)
    database_arr[path] = point_arr

CPU times: user 1min 37s, sys: 11.6 s, total: 1min 49s
Wall time: 1min 48s


## 1.2 Load the model

In [15]:
class MCClassifier(nn.Module):

    def __init__(self, pc_model, image_model, num_pc_features, num_img_features, dropout=0.5):

        super(MCClassifier, self).__init__()

        self.img_model = image_model
        self.pc_model = pc_model
        self.dropout = nn.Dropout(dropout)
        self.batch_norm = nn.BatchNorm1d(512)  # Add batch normalization layer
        self.relu = nn.ReLU()

        # Image MLP
        self.lni1 = nn.Linear(num_img_features, 512)
        self.lni2 = nn.Linear(512,256)
        #self.img_ln = nn.Linear(1280, 1024)
        
        # Point Cloud MLP
        self.lnp1 = nn.Linear(1024, 512)
        self.lnp2 = nn.Linear(512,256)
     
        
        # For scaling the point cloud features masp
        self.conv1d = nn.Conv1d(1024, 1, kernel_size=1)
    def forward(self, image, pc):
        # 2D Image
        image_features = self.img_model(image)
        img_emb = self.dropout(image_features)
        img_emb = self.lni1(img_emb)
        img_emb = self.batch_norm(img_emb)
        img_emb = self.relu(img_emb)
        #img_emb = self.lni2(img_emb)
        #img_emb = self.relu(img_emb)
        
        # Point Cloud
        pc_features = self.pc_model(pc)
        pc_features = self.conv1d(pc_features).squeeze(1)
        pc_emb = self.dropout(pc_features)
        pc_emb = self.lnp1(pc_emb)
        pc_emb = self.batch_norm(pc_emb)  # Apply batch normalization
        pc_emb = self.relu(pc_emb)
        #pc_emb = self.linear2(out)
        #pc_emb = self.relu(out)
        

        return img_emb, pc_emb

In [16]:
img_model = timm.create_model('mobilenetv2_100', pretrained=True, num_classes = 0).to('cuda')#.get_classifier()
img_model.to('cuda')
print("YES")

YES


In [17]:
pn = PointNet(emb_dims=1024, use_bn=True)
pnt_cls = Classifier(feature_model=pn)
pt_path = '/kaggle/input/3d-cls-pretrained-model/pytorch/modelnet40-pt/1/best_model.t7'
pnt_cls.load_state_dict(torch.load(pt_path, map_location='cpu'))
pc_model = pnt_cls.feature_model
pc_model.to('cuda')

PointNet(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (conv5): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
  (relu): ReLU()
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [18]:
model = MCClassifier(pc_model, img_model, 1024 ,img_model.num_features).to('cuda')

In [22]:
model = torch.load('/kaggle/input/3d-retrieval-dataset/retrival_net_(pntnet).ckpt')
model.eval()

MCClassifier(
  (img_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU6(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU6(inplace=True)
          )
          (se): Identity()
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNormAct2d(
            16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
          (dro

## 1.3 Generate prediction and write to the .csv file


In [59]:
"""import csv
test_transform = create_transform(224)

df = pd.read_csv('/kaggle/input/2d-3d-retrieval-dataset/wb_2D3Dretrieval_dataset/labels.csv')

# split the df into train and test
df['label'] = df['label'].apply(lambda x: x.split(','))
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
"""

In [60]:
test_path = 'Enter path'
test_df = pd.read_csv(test_path)

In [156]:
queries_list = os.listdir(queries_path)
queries_list = ['/kaggle/working/test_retrieval/query/' + q for q in queries_list]

In [161]:
%%time
model.eval()
with open('prediction.csv', 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        field = ["query_name", "predictions"]
        csv_writer.writerow(field)
        for q in tqdm(queries_list):
            # Load the query image
            img = Image.open(q).convert("RGB")
            img_tensor = test_transform(img).unsqueeze(0).to('cuda').float()

            # Calculate the distance between the query image and all the database images
            distances = {}
            for db in db_names:
                db_feat = torch.from_numpy(database_arr[db]).unsqueeze(0).to('cuda').float()
                img_emb, db_emb = model(img_tensor, db_feat)
                dist = torch.nn.functional.pairwise_distance(img_emb, db_emb)
                distances[db] = dist.item()

            # Sort the distances
            sorted_distances = sorted(distances.items(), key=lambda x: x[1])
            
            # Write to the output file (Top 5)
            output = [k for k, v in sorted_distances[:5]]
            
            # Turn the output in as string, seperate by a comma
            output = ', '.join(output)
            print("Output :", output)
            csv_writer.writerow([q, output])
            
            

  0%|          | 1/300 [00:02<13:59,  2.81s/it]

Output : 249.stl, 197.stl, 219.stl, 225.stl, 252.stl


  1%|          | 2/300 [00:05<13:55,  2.80s/it]

Output : 249.stl, 197.stl, 252.stl, 225.stl, 12.stl


  1%|          | 3/300 [00:08<14:02,  2.84s/it]

...

Output : 249.stl, 197.stl, 252.stl, 225.stl, 12.stl


100%|█████████▉| 299/300 [14:03<00:02,  2.80s/it]

Output : 197.stl, 249.stl, 225.stl, 12.stl, 252.stl


100%|██████████| 300/300 [14:06<00:00,  2.82s/it]

Output : 197.stl, 249.stl, 12.stl, 225.stl, 242.stl
CPU times: user 14min 15s, sys: 1.21 s, total: 14min 16s
Wall time: 14min 6s
